# Tutorial: Creating a Custom Task

TODO: words

In [1]:
import flash

import torch
from torch import nn

PyTorch version 1.7.1 available.


Creating a model. TODO: words

In [15]:
class LinearRegression(flash.Task):
    def __init__(self, num_inputs, learning_rate=0.001, metrics=None):
        # what kind of model do we want?
        model = nn.Linear(num_inputs, 1)

        # what loss function do we want?
        loss_fn = torch.nn.functional.mse_loss
        
        # what optimizer to do we want?
        optimizer = torch.optim.SGD
        
        super().__init__(
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            metrics=metrics,
            learning_rate=learning_rate,
        )
        
    def forward(self, x):
        return self.model(x)

Load data TODO: words, something about diabetes dataset (10 features, 1 output)

In [16]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [46]:
class DiabetesPipeline(flash.core.data.TaskDataPipeline):
    def after_uncollate(self, samples):
        return [f"disease progression: {float(s):.2f}" for s in samples]

class DiabetesData(flash.DataModule):
    def __init__(self, batch_size=64, num_workers=0):
        x, y = datasets.load_diabetes(return_X_y=True)
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(y).float().unsqueeze(1)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=0)

        train_ds = TensorDataset(x_train, y_train)
        test_ds = TensorDataset(x_test, y_test)
        
        super().__init__(
            train_ds=train_ds,
            test_ds=test_ds,
            batch_size=batch_size,
            num_workers=num_workers
        )
        
    @staticmethod
    def default_pipeline():
        return DiabetesPipeline()
        
        
    

Fit TODO words:

In [47]:
model = LinearRegression(num_inputs=x.shape[1])
data = DiabetesData()

trainer = flash.Trainer(max_epochs=1000)
trainer.fit(model, data)

GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | model   | Linear     | 11    
1 | metrics | ModuleDict | 0     
---------------------------------------
11        Trainable params
0         Non-trainable params
11        Total params
0.000     Total estimated model params size (MB)

  | Name    | Type       | Params
---------------------------------------
0 | model   | Linear     | 11    
1 | metrics | ModuleDict | 0     
---------------------------------------
11        Trainable params
0         Non-trainable params
11        Total params
0.000     Total estimated model params size (MB)


/usr/local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [49]:
model.predict(x_test[0:5])

['disease progression: 155.96',
 'disease progression: 156.69',
 'disease progression: 152.72',
 'disease progression: 149.06',
 'disease progression: 150.97']